In [1]:
import pandas as pd
import json

# 读取 JSONL 文件
df_text = pd.read_json('/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/stage2_1989_57/conversations.jsonl', lines=True)
df_mm = pd.read_json('/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/stage2_1989_57/graphs.jsonl', lines=True)

# # 显示 DataFrame
# print(df)

In [2]:
num_lines = len(df_text)
num_lines

1932

In [4]:
df_mm.iloc[0]['nodes']

[{'id': 0, 'content': 'clk', 'type': 'input port'},
 {'id': 1, 'content': 'rst', 'type': 'input port'},
 {'id': 2, 'content': 'a', 'type': 'input port'},
 {'id': 3, 'content': 'b', 'type': 'input port'},
 {'id': 4, 'content': 'opcode', 'type': 'input port'},
 {'id': 5, 'content': 'result', 'type': 'output port'},
 {'id': 6, 'content': 'error', 'type': 'output port'},
 {'id': 7, 'content': 'u_adder', 'type': 'submodule'},
 {'id': 8, 'content': 'u_subtractor', 'type': 'submodule'},
 {'id': 9, 'content': 'u_and', 'type': 'submodule'},
 {'id': 10, 'content': 'u_or', 'type': 'submodule'}]

In [5]:
for i in range(num_lines):
    for j in range(len(df_mm.iloc[i]['nodes'])):
        if "port" in df_mm.iloc[i]['nodes'][j]['type']:
            df_mm.iloc[i]['nodes'][j]['type'] = df_mm.iloc[i]['nodes'][j]['type'].replace(" port", "")
        df_mm.iloc[i]['nodes'][j] = df_mm.iloc[i]['nodes'][j]['content'] + " " + df_mm.iloc[i]['nodes'][j]['type']


In [11]:
df_mm.iloc[999]['nodes']

['a input',
 'b input',
 'Bin input',
 'd output',
 'Bout output',
 'sub16_inst1 submodule',
 'sub16_inst2 submodule']

In [12]:
df_mm['nodes'][100]

['clk input',
 'reset input',
 'write_en input',
 'read_reg1_sel input',
 'read_reg2_sel input',
 'write_reg_sel input',
 'write_data input',
 'read_data1 output',
 'read_data2 output',
 'u_register_array submodule',
 'u_control_unit submodule']

In [13]:
template = "Given a submodules interconnection graph: \n<graph>\n, "
# text_graph = '\n{}\n,'.format(graph)

In [14]:
new_text = []
for i in range(num_lines):
# for i in range(3):
    new_line = []
    graph = "nodes: {}, \nconnectivity: {}".format(df_mm['nodes'][i], df_mm['connectivity'][i])
    text_graph = "\n{}\n, ".format(graph)
    human_instruction = template + text_graph + df_text['Instruction'][i]
    human_instruction = {"from": "human", "value": human_instruction}
    gpt_response = df_text['Response'][i]
    gpt_response = {"from": "gpt", "value": gpt_response}
    new_line.append(human_instruction)
    new_line.append(gpt_response)
    new_line = {"conversations": new_line}
    new_line['task_id'] = df_text['task_id'][i]
    new_text.append(new_line)
    # print(json.dumps(new_line))


In [15]:
with open('/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/stage2_1989_57/without_head/cleaned_graph/conversations.json', 'w') as f:
    # for line in new_text:
    #     f.write(json.dumps(line) + '\n')
    json.dump(new_text, f)

In [16]:
new_graph_df = df_mm[['nodes', 'connectivity']]
new_graph_df
new_graph_df.to_json('/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/stage2_1989_57/without_head/cleaned_graph/graph.jsonl', orient='records', lines=True)